# Predict on Fleurs with confidence scores

In [1]:
from confidence_utils import *

from datasets import load_dataset
from IPython.display import HTML as html_print
import matplotlib.pyplot as plt

## I. Load data

In [2]:
fleurs_en = load_dataset("google/fleurs", "en_us")
fleurs_en = fleurs_en.remove_columns(['id', 'num_samples', 'path', 'gender', 'lang_id', 'language', 'lang_group_id'])
fleurs_en = fleurs_en['train']

Found cached dataset fleurs (/home/antonin/.cache/huggingface/datasets/google___fleurs/en_us/2.0.0/aabb39fb29739c495517ac904e2886819b6e344702f0a5b5283cb178b087c94a)


  0%|          | 0/3 [00:00<?, ?it/s]

## II. Predict with Wav2Vec + 4-grams

In [3]:
processor, model = load_wav2vec_model("patrickvonplaten/wav2vec2-base-960h-4-gram")

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at patrickvonplaten/wav2vec2-base-960h-4-gram and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.7/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in 'snapshot_download': allow_regex. Will not be supported from version '0.12'.

Please use `allow_patterns` and `ignore_patterns` instead.
  warnings.warn(message, FutureWarning)


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

In [5]:
fleurs_en_wav2vec = fleurs_en.map(predict_with_confidence_wav2vec, 
    fn_kwargs={"model": model, "processor": processor}, 
    remove_columns=['audio'])
fleurs_en_wav2vec

#0:   0%|          | 0/651 [00:00<?, ?ex/s]

#1:   0%|          | 0/651 [00:00<?, ?ex/s]

#2:   0%|          | 0/650 [00:00<?, ?ex/s]

#3:   0%|          | 0/650 [00:00<?, ?ex/s]

KeyboardInterrupt: 

In [ ]:
fleurs_en_wav2vec.save_to_disk(os.path.join(predictions_confidence_path, 'fleurs_en_wav2vec'))

## III. Predict with Whiser

processor_en, model_en = load_whisper_model('openai/whisper-base', 'English')

In [ ]:
model_en.config.forced_decoder_ids = processor_en.get_decoder_prompt_ids(language = "en", task = "transcribe")
fleurs_en_whisper = fleurs_en.map(predict_with_confidence_whisper, 
    fn_kwargs={"processor": processor_en, "model": model_en, "lang": "en"}, 
    batched=True, \
    remove_columns=['audio'], 
    batch_size = 1)
fleurs_en_whisper

In [ ]:
fleurs_en_whisper.save_to_disk(os.path.join(predictions_confidence_path, 'fleurs_en_whisper'))